<a href="https://colab.research.google.com/github/Animesh101-wondercar/ice_to_water/blob/main/Technical_Indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Adding Technical Indicators***
 1. MACD
 2. RSI

In [2]:
!pip install yfinance plotly pandas numpy

In [3]:
import yfinance as yf
import pandas as pd
ticker = 'RELIANCE.NS'
data = yf.download(ticker, start='2024-01-01', end='2024-05-30',interval='1d')
data_cleaned = data.drop(columns=['Adj Close']) # removing adjusted close columns
data_cleaned.fillna(method='ffill', inplace=True) # to fill those missing values
data_cleaned.dropna(inplace=True)  #to remove any rows with NAN values
data_cleaned.reset_index(inplace=True)
print(data_cleaned.head())

[*********************100%%**********************]  1 of 1 completed

        Date         Open         High          Low        Close   Volume
0 2024-01-01  2580.550049  2606.850098  2573.149902  2590.250000  2015270
1 2024-01-02  2585.000000  2615.000000  2573.000000  2611.699951  3724400
2 2024-01-03  2610.000000  2634.000000  2577.199951  2583.300049  4518768
3 2024-01-04  2588.000000  2609.850098  2579.100098  2596.649902  4806389
4 2024-01-05  2602.899902  2619.850098  2598.000000  2607.699951  4043203


In [4]:
# MACD Indicator
import plotly.graph_objects as go
ema_12 = data_cleaned['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data_cleaned['Close'].ewm(span=26, adjust=False).mean()
data_cleaned['MACD'] = ema_12 - ema_26
data_cleaned['Signal Line'] = data_cleaned['MACD'].ewm(span=9, adjust=False).mean()
fig = go.Figure()

# Closing price graph
fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Close'],
    mode='lines',
    name='Close Price'
))

# MACD line
fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['MACD'],
    mode='lines',
    name='MACD',
    yaxis='y2'
))

# Signal
fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Signal Line'],
    mode='lines',
    name='Signal Line',
    yaxis='y2'
))

# Update layout for high quality and dual y-axis
fig.update_layout(
    title='',
    xaxis_title='Date',
    yaxis_title='Stock Price',
    yaxis2=dict(
        title='MACD',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0, y=1.2),
    template='plotly_dark'
)

fig.show()


In [5]:
# RSI Indicator
def calculate_rsi(data, window):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data_cleaned['RSI'] = calculate_rsi(data_cleaned, 14)

#graph
fig = go.Figure()

# Line chart for closing price
fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Close'],
    mode='lines',
    name='Close Price'
))

# RSI line chart
fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['RSI'],
    mode='lines',
    name='RSI',
    yaxis='y2'
))

# Update layout for high quality and dual y-axis
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Stock Price',
    yaxis2=dict(
        title='RSI',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0, y=1.2),
    template='plotly_dark'
)

fig.show()

In [9]:
from plotly.subplots import make_subplots
#Bollinger Bands
def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data['Close'].rolling(window).mean()
    rolling_std = data['Close'].rolling(window).std()
    data['Bollinger Middle Band'] = rolling_mean
    data['Bollinger Upper Band'] = rolling_mean + (rolling_std * no_of_std)
    data['Bollinger Lower Band'] = rolling_mean - (rolling_std * no_of_std)
    return data

data_cleaned = calculate_bollinger_bands(data_cleaned)


fig = make_subplots(
    rows= 1, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('Price with Bollinger Bands')
)

# Line chart for closing price with Bollinger Bands
fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Close'],
    mode='lines',
    name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Bollinger Upper Band'],
    mode='lines',
    name='Bollinger Upper Band',
    line=dict(color='rgba(255,0,0,0.5)')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Bollinger Lower Band'],
    mode='lines',
    name='Bollinger Lower Band',
    line=dict(color='rgba(0,0,255,0.5)')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data_cleaned['Date'],
    y=data_cleaned['Bollinger Middle Band'],
    mode='lines',
    name='Bollinger Middle Band',
    line=dict(color='rgba(0,255,0,0.5)')
), row=1, col=1)


# Update layout for high quality
fig.update_layout(
    title='Bollinger Bands',
    xaxis_title='Date',
    yaxis_title='Stock Price',
    template='plotly_dark',
    height=1000,
    width=1500
)

fig.update_yaxes(range=[0, 100], dtick=10, row=2, col=1)

fig.show()
